In [1]:
import pandas as pd
import networkx as nx

In [11]:
df = pd.read_csv('../data_light/authors_journals.csv') \
       .drop(['Unnamed: 0','aff_name', 'aff_id','aff_city_id','author_name', 'aff_country_code'], axis=1) \
       .rename(columns={'journal.title': 'journal'}) 

# merge with pub_info to get the year
df = df.merge(pd.read_csv('../data_light/pubs_info.csv'), on='pub_id')

# merge with author_info to get the gender
df = df.merge(pd.read_csv('../data_light/authors_info.csv').drop(['current_organization_id'], axis=1), on='researcher_id')

# just a pub_id - research_id pair, no duplicated, groupby
df = df.groupby(['pub_id', 'researcher_id']).agg({'journal': 'first',
                                                  'year': 'first',
                                                  'gender': 'first',
                                                  'LMIC': 'max'}).reset_index()

# drop PLOS Medicine
df = df[df['journal'] != 'PLOS Medicine']

# exclude papers with only one author
df1 = df.groupby('pub_id').filter(lambda x: len(x) > 1) 
print(f"{len(df.researcher_id.unique())} authors (beginning)")
print(f"{len(df.researcher_id.unique()) - len(df1.researcher_id.unique())} authors dropped where there was only one author")
print(f"{len(df1.researcher_id.unique())} authors")


# exclude papers with more than 100 authors
df2 = df1.groupby('pub_id').filter(lambda x: len(x) <= 100)
print(f"{len(df1.researcher_id.unique()) - len(df2.researcher_id.unique())} authors dropped where papers had more than 100 authors")
print(f"{len(df2.researcher_id.unique())} authors")a


# drop if gender is unknown
df3 = df2[df2.gender.isna() == False]
print(f"{len(df2.researcher_id.unique()) - len(df3.researcher_id.unique())} authors dropped with no gender info")
print(f"{len(df3.researcher_id.unique())} authors (final)")

# transform female into 1, male into 0
df3['gender'] = df3['gender'].apply(lambda x: 1 if x == "female" else 0)

df = df3.copy()

df.head()

172359 authors (beginning)
7722 authors dropped with only one paper
164637 authors
2765 authors dropped with more than 100 papers
161872 authors
15739 authors dropped with no gender info
146133 authors (final)


C:\Users\João Matos\AppData\Local\Temp\ipykernel_26452\2346169848.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['gender'] = df3['gender'].apply(lambda x: 1 if x == "female" else 0)


,pub_id,researcher_id,journal,year,gender,LMIC
0,pub.1000001707,ur.01006172666.33,JAMA,2015,1,0
1,pub.1000001707,ur.01012736025.78,JAMA,2015,0,0
2,pub.1000001707,ur.010132635727.81,JAMA,2015,0,0
3,pub.1000001707,ur.01023477132.25,JAMA,2015,1,0
4,pub.1000001707,ur.01043100547.54,JAMA,2015,1,0


In [12]:
# count na 
df3.isna().sum()

pub_id           0
researcher_id    0
journal          0
year             0
gender           0
LMIC             0
dtype: int64

In [13]:
# count the number of authors in total
print(f"Total number of authors: {len(df.researcher_id.unique())}")
print(f"Total number of publications: {len(df.pub_id.unique())}")

Total number of authors: 146133
Total number of publications: 50149


In [24]:
# count the number of authors per publication, sort desc
af = df.groupby('pub_id') \
       .agg({'researcher_id': 'count'}) \
       .reset_index() \
       .sort_values('researcher_id', ascending=False)

# print median and IQR of the number of authors per publication
print(f"Median number of authors per publication: {af.researcher_id.median()}")
print(f"IQR of the number of authors per publication: {af.researcher_id.quantile(0.25) , af.researcher_id.quantile(0.75)}")


Median number of authors per publication: 3.0
IQR of the number of authors per publication: (2.0, 6.0)


In [39]:
# print the number of publications per year, in bins of 4 years
# 2007-2010, 2011-2014, 2015-2018, 2019-2022
# drop 2023
df = df[df.year != 2023]
yf = df.year.value_counts().sort_index()
# sum of publications in bins of 4 years
print(f"Number of publications in 2007-2010: {yf[2007] + yf[2008] + yf[2009] + yf[2010]}")
print(f"Number of publications in 2011-2014: {yf[2011] + yf[2012] + yf[2013] + yf[2014]}")
print(f"Number of publications in 2015-2018: {yf[2015] + yf[2016] + yf[2017] + yf[2018]}")
print(f"Number of publications in 2019-2022: {yf[2019] + yf[2020] + yf[2021] + yf[2022]}")


Number of publications in 2007-2010: 43241
Number of publications in 2011-2014: 58408
Number of publications in 2015-2018: 83173
Number of publications in 2019-2022: 95192


In [40]:
print(f"% of publications in 2007-2010: {(yf[2007] + yf[2008] + yf[2009] + yf[2010])/yf.sum()*100}")
print(f"% of publications in 2011-2014: {(yf[2011] + yf[2012] + yf[2013] + yf[2014])/yf.sum()*100}")
print(f"% of publications in 2015-2018: {(yf[2015] + yf[2016] + yf[2017] + yf[2018])/yf.sum()*100}")
print(f"% of publications in 2019-2022: {(yf[2019] + yf[2020] + yf[2021] + yf[2022])/yf.sum()*100}")

% of publications in 2007-2010: 15.442442163606104
% of publications in 2011-2014: 20.858957052147392
% of publications in 2015-2018: 29.703157699257893
% of publications in 2019-2022: 33.99544308498861


In [44]:
# print the number of publications per journal
jf = df.groupby('journal').agg({'pub_id': 'count'}).sort_values('pub_id', ascending=False)
print(jf)

                                 pub_id
journal                                
The Lancet                        88902
JAMA                              53010
The BMJ                           49332
Nature Medicine                   45054
New England Journal of Medicine   43716


In [45]:
# print the % of publications per journal
print(jf/jf.sum()*100)  

                                    pub_id
journal                                   
The Lancet                       31.749127
JAMA                             18.931196
The BMJ                          17.617691
Nature Medicine                  16.089910
New England Journal of Medicine  15.612077


In [48]:

import numpy as np
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [59]:
# get the median and IQR of the number of authors per publication per journal
apf = df.groupby(['pub_id', 'journal']) \
         .agg({'researcher_id': 'count'}) \
            .reset_index() \
            .groupby('journal') \
            .agg({'researcher_id': ['median', percentile(25), percentile(75)]}) \
            .reset_index()

apf

journal researcher_id                            
                                          median percentile_25 percentile_75
0                             JAMA           3.0           2.0           6.0
1                  Nature Medicine          10.0           3.0          17.0
2  New England Journal of Medicine           3.0           2.0           5.0
3                          The BMJ           3.0           2.0           5.0
4                       The Lancet           3.0           2.0           6.0

In [56]:
# create new column in df with the year bins
df['year_bin'] = df['year'].apply(lambda x: '2007-2010' if x in [2007, 2008, 2009, 2010] else \
                                             '2011-2014' if x in [2011, 2012, 2013, 2014] else \
                                                '2015-2018' if x in [2015, 2016, 2017, 2018] else \
                                                    '2019-2022')

In [60]:
# get the median and IQR of the number of authors per publication per bin of 4 years
apyf = df.groupby(['pub_id', 'year_bin']) \
            .agg({'researcher_id': 'count'}) \
            .reset_index() \
            .groupby('year_bin') \
            .agg({'researcher_id': ['median', percentile(25), percentile(75)]}) \
            .reset_index()

apyf


year_bin researcher_id                            
                    median percentile_25 percentile_75
0  2007-2010           3.0           2.0           6.0
1  2011-2014           3.0           2.0           6.0
2  2015-2018           3.0           2.0           6.0
3  2019-2022           3.0           2.0           7.0

In [61]:
# print number of LMIC authors
print(df.LMIC.value_counts())
print(df.LMIC.value_counts(normalize=True))

0    256308
1     23706
Name: LMIC, dtype: int64
0    0.91534
1    0.08466
Name: LMIC, dtype: float64


In [71]:
# print number of publications with at least one LMIC author
print(df.groupby('pub_id').agg({'LMIC': 'max'}).LMIC.value_counts())
print(df.groupby('pub_id').agg({'LMIC': 'max'}).LMIC.value_counts(normalize=True))

0    42666
1     7376
Name: LMIC, dtype: int64
0    0.852604
1    0.147396
Name: LMIC, dtype: float64


In [62]:
# print number of female authors with value counts
print(df.gender.value_counts())
print(df.gender.value_counts(normalize=True))

0    186709
1     93305
Name: gender, dtype: int64
0    0.666785
1    0.333215
Name: gender, dtype: float64


In [72]:
# print publications with at least one female author
print(df.groupby('pub_id').agg({'gender': 'max'}).gender.value_counts())
print(df.groupby('pub_id').agg({'gender': 'max'}).gender.value_counts(normalize=True))

1    33633
0    16409
Name: gender, dtype: int64
1    0.672095
0    0.327905
Name: gender, dtype: float64


In [67]:
# get number of authors per year bin
ybaf = df.groupby('year_bin').agg({'researcher_id': 'count'}).reset_index()
ybaf

,year_bin,researcher_id
0,2007-2010,43241
1,2011-2014,58408
2,2015-2018,83173
3,2019-2022,95192


In [69]:
# same now normalized by the number of authors in total
ybaf['researcher_id']/ybaf.researcher_id.sum()*100


0    15.442442
1    20.858957
2    29.703158
3    33.995443
Name: researcher_id, dtype: float64

In [75]:
# number of authors per journal
jaf = df.groupby('journal').agg({'researcher_id': 'count'}).reset_index()
jaf

,journal,researcher_id
0,JAMA,53010
1,Nature Medicine,45054
2,New England Journal of Medicine,43716
3,The BMJ,49332
4,The Lancet,88902


In [74]:
jaf.researcher_id/jaf.researcher_id.sum()*100

0    18.931196
1    16.089910
2    15.612077
3    17.617691
4    31.749127
Name: researcher_id, dtype: float64

In [20]:
df.to_csv('../data_light/data_clean.csv', index=False)

In [22]:
df.head()

,pub_id,researcher_id,journal,year,gender,LMIC
0,pub.1000001707,ur.01006172666.33,JAMA,2015,1,0
1,pub.1000001707,ur.01012736025.78,JAMA,2015,0,0
2,pub.1000001707,ur.010132635727.81,JAMA,2015,0,0
3,pub.1000001707,ur.01023477132.25,JAMA,2015,1,0
4,pub.1000001707,ur.01043100547.54,JAMA,2015,1,0


In [ ]:
# create tableone
